In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
dbutils.fs.ls("/FileStore/shared_uploads/noelroberttemp@outlook.com/")

Out[3]: [FileInfo(path='dbfs:/FileStore/shared_uploads/noelroberttemp@outlook.com/Customers.csv', name='Customers.csv', size=19786, modificationTime=1727336006000),
 FileInfo(path='dbfs:/FileStore/shared_uploads/noelroberttemp@outlook.com/sales_data_sample.csv', name='sales_data_sample.csv', size=527958, modificationTime=1727091256000)]

dbutils.fs.ls(path) -> filenames list
dbutils.fs.rm(path) -> remove a file

In [0]:
df = spark.read.format('csv').option('header', 'true').option('inferschema', 'true').load('dbfs:/FileStore/shared_uploads/noelroberttemp@outlook.com/sales_data_sample.csv')

In [0]:
df.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

In [0]:
df.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [0]:
from pyspark.sql.functions import to_timestamp, col, dayofweek

df = df.withColumn('ORDERDATE', to_timestamp(col('ORDERDATE'), 'M/d/yyyy H:mm'))
df = df.withColumn('DAY_OF_WEEK', dayofweek(col('ORDERDATE')))

In [0]:
df.createOrReplaceTempView('productSales')

# 1. Summary of Product Sales

In [0]:
%sql
SELECT COUNT(*) as sales_count FROM productSales

sales_count
2823


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT ROUND(SUM(SALES), 2) AS total_sales FROM productSales

total_sales
1.003262885E7


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT ROUND(SUM(SALES - PRICEEACH*QUANTITYORDERED), 2) AS total_profit FROM productSales

total_profit
1741742.06


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT COUNT(DISTINCT CUSTOMERNAME) AS total_customers FROM productSales

total_customers
92


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT COUNT(DISTINCT PRODUCTCODE) AS total_products FROM productSales

total_products
109


Databricks visualization. Run in Databricks to view.

# 2. Quantity Sold per Product

In [0]:
%sql
SELECT PRODUCTLINE, SUM(QUANTITYORDERED) AS ProductsSold FROM productSales
GROUP BY PRODUCTLINE
ORDER BY PRODUCTLINE

PRODUCTLINE,ProductsSold
Classic Cars,33992
Motorcycles,11663
Planes,10727
Ships,8127
Trains,2712
Trucks and Buses,10777
Vintage Cars,21069


Databricks visualization. Run in Databricks to view.

# 3. Quantity Sold per Month

In [0]:
%sql
SELECT MONTH_ID, SUM(QUANTITYORDERED) AS ProductsSold,
      CASE
          WHEN MONTH_ID = 1 THEN 'January'
          WHEN MONTH_ID = 2 THEN 'February'
          WHEN MONTH_ID = 3 THEN 'March'
          WHEN MONTH_ID = 4 THEN 'April'
          WHEN MONTH_ID = 5 THEN 'May'
          WHEN MONTH_ID = 6 THEN 'June'
          WHEN MONTH_ID = 7 THEN 'July'
          WHEN MONTH_ID = 8 THEN 'August'
          WHEN MONTH_ID = 9 THEN 'September'
          WHEN MONTH_ID = 10 THEN 'October'
          WHEN MONTH_ID = 11 THEN 'November'
          WHEN MONTH_ID = 12 THEN 'December'
      END AS month_name
FROM productSales
GROUP BY MONTH_ID
ORDER BY MONTH_ID

MONTH_ID,ProductsSold,month_name
1,7997,January
2,7903,February
3,7585,March
4,6704,April
5,8992,May
6,4620,June
7,4899,July
8,6538,August
9,5681,September
10,10998,October


Databricks visualization. Run in Databricks to view.

# 4. Month-on-month Sales growth

In [0]:
%sql
SELECT CAST(DAY(ORDERDATE) AS int) AS day, CAST(MONTH(ORDERDATE) AS int) AS month,
        SUM(QUANTITYORDERED) AS ProductsSold
FROM productSales
GROUP BY month, day
ORDER BY month, day

day,month,ProductsSold
2,1,530
5,1,117
6,1,707
7,1,96
9,1,411
10,1,197
12,1,1046
15,1,530
16,1,612
19,1,183


Databricks visualization. Run in Databricks to view.

# 5. Weekly Sales growth

In [0]:
%sql
SELECT DAY_OF_WEEK, CAST(MONTH(ORDERDATE) AS int) AS month,
        CASE
            WHEN DAY_OF_WEEK=1 THEN 'Sunday'
            WHEN DAY_OF_WEEK=2 THEN 'Monday'
            WHEN DAY_OF_WEEK=3 THEN 'Tuesday'
            WHEN DAY_OF_WEEK=4 THEN 'Wednesday'
            WHEN DAY_OF_WEEK=5 THEN 'Thursday'
            WHEN DAY_OF_WEEK=6 THEN 'Friday'
            WHEN DAY_OF_WEEK=7 THEN 'Saturday'
        END as DAY_NAME,
        SUM(Sales) AS TotalSales
FROM productSales
GROUP BY month, DAY_OF_WEEK
ORDER BY month, DAY_OF_WEEK

DAY_OF_WEEK,month,DAY_NAME,TotalSales
1,1,Sunday,36124.270000000004
2,1,Monday,153302.92
4,1,Wednesday,166398.41000000003
5,1,Thursday,223950.17
6,1,Friday,206098.66999999995
1,2,Sunday,50432.55
2,2,Monday,92822.98000000001
3,2,Tuesday,120226.53000000001
4,2,Wednesday,147240.6
5,2,Thursday,329903.5399999999


Databricks visualization. Run in Databricks to view.

# 6. Top 20 cities with Most Quantity Orders

In [0]:
%sql
SELECT CITY, SUM(QUANTITYORDERED) AS TotalQuantityOrdered
FROM productSales
GROUP BY CITY
ORDER BY TotalQuantityOrdered DESC
LIMIT 20

CITY,TotalQuantityOrdered
Madrid,10958
San Rafael,6366
NYC,5294
Singapore,2760
Paris,2521
San Francisco,2139
Nantes,2102
New Bedford,2043
Melbourne,1926
Manchester,1778


Databricks visualization. Run in Databricks to view.

# 7. Customer Sales Contribution

In [0]:
%sql
SELECT CUSTOMERNAME, ROUND(SUM(SALES), 3) AS sum_sales,
        ROUND(SUM(SALES)*100/(SELECT SUM(SALES) FROM productSales), 4) AS contribution_percent
FROM productSales
GROUP BY CUSTOMERNAME
ORDER BY sum_sales DESC
LIMIT 10

CUSTOMERNAME,sum_sales,contribution_percent
Euro Shopping Channel,912294.11,9.0933
Mini Gifts Distributors Ltd.,654858.06,6.5273
"Australian Collectors, Co.",200995.41,2.0034
Muscle Machine Inc,197736.94,1.9709
La Rochelle Gifts,180124.9,1.7954
"Dragon Souveniers, Ltd.",172989.68,1.7243
Land of Toys Inc.,164069.44,1.6354
The Sharp Gifts Warehouse,160010.27,1.5949
"AV Stores, Co.",157807.81,1.5729
"Anna's Decorations, Ltd",153996.13,1.535


Databricks visualization. Run in Databricks to view.

# 8. Seasonal Trends

In [0]:
# spark.sql("""
#     SELECT MONTH_ID,
#           CASE
#             WHEN QTR_ID=1 THEN 'Winter'
#             WHEN QTR_ID=2 THEN 'Spring'
#             WHEN QTR_ID=3 THEN 'Summer'
#             WHEN QTR_ID=4 THEN 'Fall'
#           END AS season,
#           ROUND(SUM(SALES), 3) AS TotalSales
#     FROM productSales
#     GROUP BY MONTH_ID, QTR_ID
#     ORDER BY MONTH_ID ASC
# """).show()

In [0]:
%sql
SELECT MONTH_ID,
      CASE
        WHEN QTR_ID=1 THEN 'Winter'
        WHEN QTR_ID=2 THEN 'Spring'
        WHEN QTR_ID=3 THEN 'Summer'
        WHEN QTR_ID=4 THEN 'Fall'
      END AS season,
      ROUND(SUM(SALES), 3) AS TotalSales,
      CASE
              WHEN MONTH_ID=1 THEN 'January'
              WHEN MONTH_ID=2 THEN 'February'
              WHEN MONTH_ID=3 THEN 'March'
              WHEN MONTH_ID=4 THEN 'April'
              WHEN MONTH_ID=5 THEN 'May'
              WHEN MONTH_ID=6 THEN 'June'
              WHEN MONTH_ID=7 THEN 'July'
              WHEN MONTH_ID=8 THEN 'August'
              WHEN MONTH_ID=9 THEN 'September'
              WHEN MONTH_ID=10 THEN 'October'
              WHEN MONTH_ID=11 THEN 'November'
              WHEN MONTH_ID=12 THEN 'December'
       END AS month_name
FROM productSales
GROUP BY MONTH_ID, QTR_ID
ORDER BY MONTH_ID ASC

MONTH_ID,season,TotalSales,month_name
1,Winter,785874.44,January
2,Winter,810441.9,February
3,Winter,754501.39,March
4,Spring,669390.96,April
5,Spring,923972.56,May
6,Spring,454756.78,June
7,Summer,514875.97,July
8,Summer,659310.57,August
9,Summer,584724.27,September
10,Fall,1121215.22,October


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# 9. Categorize PRODUCTLINE as 'Hot' or 'Cold' based on ORDERQUANTITY in each quarter using CASE WHEN

In [0]:
%sql
SELECT PRODUCTLINE, quantity, QTR_ID,
CASE
  WHEN quantity=max THEN 'Hot'
  WHEN quantity=min THEN 'Cold'
  else 'medium'
END AS category
FROM (
    SELECT QTR_ID, PRODUCTLINE,
          SUM(QUANTITYORDERED) AS quantity,
          MAX(SUM(QUANTITYORDERED)) OVER(PARTITION BY QTR_ID) AS max,
          MIN(SUM(QUANTITYORDERED)) OVER(PARTITION BY QTR_ID) AS min
    FROM productSales
    GROUP BY QTR_ID, PRODUCTLINE
) AS _
WHERE quantity = max OR quantity = min
ORDER BY QTR_ID, quantity DESC

PRODUCTLINE,quantity,QTR_ID,category
Classic Cars,7784,1,Hot
Trains,693,1,Cold
Classic Cars,6447,2,Hot
Trains,439,2,Cold
Classic Cars,6190,3,Hot
Trains,531,3,Cold
Classic Cars,13571,4,Hot
Trains,1049,4,Cold


Databricks visualization. Run in Databricks to view.

# 10. Identify products with outlier sales in each quarter

In [0]:
%sql
SELECT PRODUCTLINE, SALES, QTR_ID, YEAR_ID,
      PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY SALES) OVER (PARTITION BY QTR_ID) AS Q1,
      PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY SALES) OVER (PARTITION BY QTR_ID) AS Q3,
      CASE
          WHEN (SALES > Q3+1.5*(Q3-Q1)) OR (SALES < Q1-1.5*(Q3-Q1)) THEN 1
          ELSE 0
        END AS isOutlier
FROM productSales

PRODUCTLINE,SALES,QTR_ID,YEAR_ID,Q1,Q3,isOutlier
Motorcycles,2871.0,1,2003,2231.0,4517.91,0
Motorcycles,4708.44,1,2004,2231.0,4517.91,0
Motorcycles,3965.66,1,2004,2231.0,4517.91,0
Motorcycles,733.11,1,2005,2231.0,4517.91,0
Motorcycles,3207.12,1,2005,2231.0,4517.91,0
Classic Cars,5404.62,1,2003,2231.0,4517.91,0
Classic Cars,7209.11,1,2003,2231.0,4517.91,0
Classic Cars,6075.3,1,2004,2231.0,4517.91,0
Classic Cars,6463.23,1,2004,2231.0,4517.91,0
Classic Cars,4514.92,1,2005,2231.0,4517.91,0


Databricks visualization. Run in Databricks to view.

# 11. Monthly sales Performance by each Territory

In [0]:
%sql
SELECT TERRITORY, MONTH_ID, 
       ROUND(SUM(SALES - PRICEEACH*QUANTITYORDERED), 3) AS PROFIT,
       ROUND(PROFIT*100/SUM(SALES), 5) AS PROFIT_PERCENTAGE,
       CASE
              WHEN MONTH_ID=1 THEN 'January'
              WHEN MONTH_ID=2 THEN 'February'
              WHEN MONTH_ID=3 THEN 'March'
              WHEN MONTH_ID=4 THEN 'April'
              WHEN MONTH_ID=5 THEN 'May'
              WHEN MONTH_ID=6 THEN 'June'
              WHEN MONTH_ID=7 THEN 'July'
              WHEN MONTH_ID=8 THEN 'August'
              WHEN MONTH_ID=9 THEN 'September'
              WHEN MONTH_ID=10 THEN 'October'
              WHEN MONTH_ID=11 THEN 'November'
              WHEN MONTH_ID=12 THEN 'December'
       END AS month_name
FROM productSales
GROUP BY TERRITORY, MONTH_ID
ORDER BY TERRITORY, MONTH_ID

TERRITORY,MONTH_ID,PROFIT,PROFIT_PERCENTAGE,month_name
APAC,1,7920.88,26.53368,January
APAC,2,13648.46,14.27855,February
APAC,3,9550.25,27.16207,March
APAC,4,7065.31,13.33449,April
APAC,5,6359.12,9.3396,May
APAC,6,8336.98,18.2075,June
APAC,7,16646.06,20.29284,July
APAC,9,17502.65,17.37496,September
APAC,11,37671.24,19.02671,November
APAC,12,3116.87,8.22281,December


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# 12. Identify consecutive quarters of sales decline for each product

In [0]:
%sql
SELECT *, LAG(Total_Quarterly_Sales, 1, 0) OVER(PARTITION BY PRODUCTLINE ORDER BY PRODUCTLINE, QTR_ID) AS prev_qtr_sales,
          (Total_Quarterly_Sales-prev_qtr_sales) AS difference,
          ROUND(difference*100/Total_Quarterly_Sales, 3) AS difference_percentage
FROM (
    SELECT PRODUCTLINE, QTR_ID, 
          ROUND(SUM(SALES), 3) AS Total_Quarterly_Sales
    FROM productSales
    GROUP BY PRODUCTLINE, QTR_ID
    ORDER BY PRODUCTLINE, QTR_ID
) AS _

PRODUCTLINE,QTR_ID,Total_Quarterly_Sales,prev_qtr_sales,difference,difference_percentage
Classic Cars,1,880278.9,0.0,880278.9,100.0
Classic Cars,2,766618.73,880278.9,-113660.17000000004,-14.826
Classic Cars,3,745268.46,766618.73,-21350.27000000002,-2.865
Classic Cars,4,1527449.57,745268.46,782181.1100000001,51.208
Motorcycles,1,264385.54,0.0,264385.54,100.0
Motorcycles,2,277822.27,264385.54,13436.73000000004,4.836
Motorcycles,3,213194.7,277822.27,-64627.57000000001,-30.314
Motorcycles,4,410985.83,213194.7,197791.13,48.126
Planes,1,234356.51,0.0,234356.51,100.0
Planes,2,247054.75,234356.51,12698.23999999999,5.14


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# 13. median sales for each product line in every quarter

In [0]:
%sql
SELECT PRODUCTLINE, QTR_ID, ROUND(MAX(Median), 3) AS Median_Value
FROM (
    SELECT PRODUCTLINE, QTR_ID, SALES, 
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY SALES) OVER (PARTITION BY PRODUCTLINE, QTR_ID) as Median
    FROM productSales
) AS _
group BY PRODUCTLINE, QTR_ID
ORDER BY PRODUCTLINE, QTR_ID


PRODUCTLINE,QTR_ID,Median_Value
Classic Cars,1,3970.56
Classic Cars,2,3922.56
Classic Cars,3,3754.05
Classic Cars,4,3609.36
Motorcycles,1,3214.56
Motorcycles,2,2958.34
Motorcycles,3,3095.19
Motorcycles,4,3199.98
Planes,1,2951.49
Planes,2,2719.31


Databricks visualization. Run in Databricks to view.

# 14. Percent rank of Products based on Gross sales amount in each quarter

In [0]:
%sql
SELECT PRODUCTLINE, QTR_ID, GrossSales, PERCENT_RANK() OVER(PARTITION BY QTR_ID ORDER BY GrossSales DESC) AS percent_rank
FROM (
  SELECT PRODUCTLINE, QTR_ID, ROUND(SUM(SALES), 3) AS GrossSales
  FROM productSales
  GROUP BY QTR_ID, PRODUCTLINE
  ORDER BY QTR_ID, PRODUCTLINE
) AS _

PRODUCTLINE,QTR_ID,GrossSales,percent_rank
Classic Cars,1,880278.9,0.0
Vintage Cars,1,501184.64,0.16666666666666666
Motorcycles,1,264385.54,0.3333333333333333
Planes,1,234356.51,0.5
Trucks and Buses,1,208619.04,0.6666666666666666
Ships,1,203877.3,0.8333333333333334
Trains,1,58115.8,1.0
Classic Cars,2,766618.73,0.0
Vintage Cars,2,337386.98,0.16666666666666666
Motorcycles,2,277822.27,0.3333333333333333


Databricks visualization. Run in Databricks to view.

# 15. categorize product category sales performance as 'High', 'Medium' and 'Low' based on percentile

In [0]:
%sql
-- SELECT PRODUCTLINE, SALES,
--        NTILE(3) OVER (PARTITION BY PRODUCTLINE ORDER BY SALES) AS ntile,
--        CASE
--            WHEN ntile=1 THEN 'Low'
--            WHEN ntile=2 THEN 'Medium'
--            WHEN ntile=3 THEN 'High'
--        END AS sales_performance_category
-- FROM productSales

SELECT PRODUCTLINE, total_sales_per_prodline, NTILE(3) OVER (ORDER BY total_sales_per_prodline) AS ntile,
        CASE
            WHEN ntile=1 THEN 'Low'
            WHEN ntile=2 THEN 'Medium'
            WHEN ntile=3 THEN 'High'
        END AS sales_performance_category
FROM (
    SELECT PRODUCTLINE, ROUND(SUM(SALES), 3) AS total_sales_per_prodline
    FROM productSales
    GROUP BY PRODUCTLINE
) AS _;

PRODUCTLINE,total_sales_per_prodline,ntile,sales_performance_category
Trains,226243.47,1,Low
Ships,714437.13,1,Low
Planes,975003.57,1,Low
Trucks and Buses,1127789.84,2,Medium
Motorcycles,1166388.34,2,Medium
Vintage Cars,1903150.84,3,High
Classic Cars,3919615.66,3,High


Databricks visualization. Run in Databricks to view.

# 16. Rank 5 customers by total purchase in every quarter of each year

In [0]:
%sql
SELECT CUSTOMERNAME, YEAR_ID, QTR_ID, Total_Purchase, rank,
       CONCAT(YEAR_ID, ',', QTR_ID) AS year_and_quarter
FROM (
  SELECT CUSTOMERNAME, YEAR_ID, QTR_ID, ROUND(SUM(QUANTITYORDERED * PRICEEACH), 3) AS Total_Purchase,
        DENSE_RANK() OVER(PARTITION BY YEAR_ID, QTR_ID ORDER BY SUM(QUANTITYORDERED*PRICEEACH) DESC) AS rank
  FROM productSales
  GROUP BY CUSTOMERNAME, YEAR_ID, QTR_ID
  ORDER BY YEAR_ID, QTR_ID
) AS _
WHERE rank <= 5

CUSTOMERNAME,YEAR_ID,QTR_ID,Total_Purchase,rank,year_and_quarter
Rovelli Gifts,2003,1,52286.77,1,"2003,1"
Baane Mini Imports,2003,1,47886.21,2,"2003,1"
Cruz & Sons Co.,2003,1,47185.22,3,"2003,1"
Danish Wholesale Imports,2003,1,46614.17,4,"2003,1"
"AV Stores, Co.",2003,1,43515.07,5,"2003,1"
"Corrida Auto Replicas, Ltd",2003,2,52927.75,1,"2003,2"
"Australian Collectors, Co.",2003,2,50117.99,2,"2003,2"
Muscle Machine Inc,2003,2,46314.64,3,"2003,2"
Marseille Mini Autos,2003,2,42888.13,4,"2003,2"
Euro Shopping Channel,2003,2,37092.61,5,"2003,2"


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# 17. Identify and Present the Trends of Sales Growth by Month. [bar graph, in percentage form]

In [0]:
%sql
SELECT MONTH_ID, ROUND(SUM(SALES), 3) AS MonthlySales,
       LAG(ROUND(SUM(SALES), 3), 1, 0) OVER(ORDER BY MONTH_ID) AS Prev_MonthlySales,
       ROUND((MonthlySales-Prev_MonthlySales)*100/MonthlySales, 3) AS PercentGrowth,
       CASE
              WHEN MONTH_ID=1 THEN 'January'
              WHEN MONTH_ID=2 THEN 'February'
              WHEN MONTH_ID=3 THEN 'March'
              WHEN MONTH_ID=4 THEN 'April'
              WHEN MONTH_ID=5 THEN 'May'
              WHEN MONTH_ID=6 THEN 'June'
              WHEN MONTH_ID=7 THEN 'July'
              WHEN MONTH_ID=8 THEN 'August'
              WHEN MONTH_ID=9 THEN 'September'
              WHEN MONTH_ID=10 THEN 'October'
              WHEN MONTH_ID=11 THEN 'November'
              WHEN MONTH_ID=12 THEN 'December'
       END AS month_name
FROM productSales 
GROUP BY MONTH_ID
ORDER BY MONTH_ID

MONTH_ID,MonthlySales,Prev_MonthlySales,PercentGrowth,month_name
1,785874.44,0.0,100.0,January
2,810441.9,785874.44,3.031,February
3,754501.39,810441.9,-7.414,March
4,669390.96,754501.39,-12.715,April
5,923972.56,669390.96,27.553,May
6,454756.78,923972.56,-103.18,June
7,514875.97,454756.78,11.676,July
8,659310.57,514875.97,21.907,August
9,584724.27,659310.57,-12.756,September
10,1121215.22,584724.27,47.849,October


Databricks visualization. Run in Databricks to view.